In [107]:
%pylab inline
%config IPCompleter.greedy=True
import numpy as np
from numpy.random import dirichlet, multinomial
import sys

np.random.seed(6269)


Populating the interactive namespace from numpy and matplotlib


In [108]:
def visualizeTopics(model, id2word, n_top_words=12):
    for i, topic in enumerate(model.components_):
        print "Topic {}:".format(i+1)
        print " ".join([id2word[j] 
                        for j in topic.argsort()[:-n_top_words - 1:-1]])

In [109]:
""" Generate toy documents """
K = 3
D = 1000
N = 10
V = 12

# 1) Set alpha and beta for generative model
alpha = np.array([2, 2, 2])
beta = np.array([[0.25, 0.25, 0.25, 0.25, 0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0.25, 0.25, 0.25, 0.25, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0 ,0, 0.25, 0.25, 0.25, 0.25]])

# 2) Using alpha, sample theta_d for each of the D documents
theta = dirichlet(alpha, D)

# 3) Sample topic z_dn for each of the N words of each of the D documents
# 4) Sample word w_dn from topic beta_k corresponding to z_dn
docs = []
for d in range(D):
    # Counts of words for each topic in doc d (e.g [4, 3, 3] for N = 10)
    z_d = multinomial(N, theta[d])
    # One hot matrix of words for doc d
    w_d = []
    for k in xrange(K):
        for i in xrange(z_d[k]):
            w_d.append(multinomial(1, beta[k]))
    docs.append(np.array(w_d))
    

In [110]:
class LDA_gibbs:

    def __init__(self, n_topics, id2word=None):
        """
        Parameters
        ----------
        n_topics : int
            number of topics for the model
        id2word : dictionary
            used to map id of word to real world
            helps to monitor progress of learning
        """
        
        self.n_topics = n_topics
        # Initialize alpha
        self.alpha = np.ones(self.n_topics) / self.n_topics
        self.id2word = id2word
        
    def fit(self, docs, threshold=0.001):
        """
        Learning the parameters for the model
        
        Parameters
        ----------
        docs : list of array-like elements, length = n_docs
            each element is an array of dimensions n_words
            by number of words in vocabulary (n_vocab). The 
            number of words varies from one text to the other.
        threshold : float
            used to stop fitting when the model has converged
            to a final state.
        """
        
        self.n_docs = len(docs)
        self.n_vocab = docs[0].shape[1]
        self.beta = np.ones(self.n_vocab) / self.n_vocab
        
        #initializing counters
        self._initialize_counters(docs)
    
        old_loss = 10
        new_loss = 0
        
        for i in xrange(100):
            
            if i%10 == 0:
                #monitoring progress by showing top words for each topic
                print i
                self._estimate_parameters()
                if id2word:
                    visualizeTopics(self, id2word)

            self._gibbs_sampling_fit(docs)

        #update parameters at the end of training
        self._estimate_parameters()
    
    def _initialize_counters(self, docs):
        """
        Initializing counters randomly
        """
        
        #initialize arrays to 0
        self.topic_doc_count = np.zeros((self.n_docs, self.n_topics))
        self.term_topic_count = np.zeros((self.n_topics, self.n_vocab))
        self.phi = np.zeros_like(self.topic_doc_count)
        self.theta = np.zeros_like(self.topic_doc_count)
        
        #keep topic assignment for each word in memory
        self.topic_assign = []
    
        for i in xrange(self.n_docs):
            
            n_words = len(docs[i])

            #sample topic randomly for each word in document
            topic = np.random.randint(self.n_topics, size=n_words)
            self.topic_assign.append(topic)
            
            #count number of words belonging to each topic
            self.topic_doc_count[i] = np.bincount(topic, minlength=self.n_topics)
            
            #count word occurence for each topic
            self.term_topic_count += np.array([np.sum(docs[i][np.where(topic==j)], axis=0)\
                                               for j in range(self.n_topics)])
    
    def _gibbs_sampling_fit(self, docs):
        """
        Update counters for each word using gibbs sampling
        """
        
        for i in xrange(self.n_docs):
            
            n_words = len(docs[i])
            
            for j in xrange(n_words):
                
                #decrement associated counters
                self.topic_doc_count[i][self.topic_assign[i][j]] -= 1
                self.term_topic_count[self.topic_assign[i][j]] -= docs[i][j]
                
                #sample new topic
                word_idx = np.argmax(docs[i][j])               
                prob = (self.topic_doc_count[i] + self.alpha) * \
                        (self.term_topic_count[:, word_idx] + self.beta[word_idx]) /\
                        (np.sum(self.term_topic_count, axis=1) + np.sum(self.beta))                
                prob /= np.sum(prob)                      #normalize
                prob = np.cumsum(prob)                    #cdf
                self.topic_assign[i][j] = np.argmax(prob > np.random.random())

                #increment associated counters
                self.topic_doc_count[i][self.topic_assign[i][j]] += 1
                self.term_topic_count[self.topic_assign[i][j]] += docs[i][j]      
                
    def _estimate_parameters(self):
        """
        Estimate parameters used to make predictions
        """
        
        self.phi = np.divide((self.term_topic_count + self.beta).T, 
                             np.sum((self.term_topic_count + self.beta), axis=1)).T
        
        self.theta = np.divide((self.topic_doc_count + self.alpha).T, 
                             np.sum((self.topic_doc_count + self.alpha), axis=1)).T
        
        self.components_ = self.phi
        

In [111]:
""" Get the 20 news groups data """
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import pickle

cats = ['sci.space', 'rec.sport.hockey']
newsgroups = fetch_20newsgroups(shuffle=True, random_state=1, subset="train",
                                remove=("headers", "footers", "quotes"), categories=cats)

In [112]:
""" Prepare input for sklearn (counts) """
n_features = 1000
vectorizer = CountVectorizer(max_features=n_features, stop_words="english")

# Word counts per document matrix (input for sklearn)
W_counts = vectorizer.fit_transform(newsgroups.data)

# Keep track of vocabulary to visualize top words of each topic
vocabulary = vectorizer.get_feature_names()

In [113]:
""" Prepare input for our model (one hot vectors) """
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from gensim.corpora import Dictionary
import string

# 1) Tokenize document
# 2) Remove stop words
# 3) Lemmatize
lemmatizer = WordNetLemmatizer()
#stemmer = PorterStemmer()
# TODO : treat special list better
# TODO See what count vectorizer does
special = ["''", "'s", "``", "n't", "...", "--"]
stop = set(stopwords.words('english') + \
           list(string.punctuation) + 
           list(string.digits) + 
           list(string.ascii_lowercase) + \
           special)

def prepare_document(doc):
    words = [lemmatizer.lemmatize(w) for w in word_tokenize(doc.lower()) 
             if w not in stop] 
    return words


# List of documents (each document is a list of word tokens)
texts = [prepare_document(text) for text in newsgroups.data]

# Create a gensim dictionary for our corpus
dic = Dictionary(texts)

# Keep only k most frequent words
n_features = 1000
dic.filter_extremes(keep_n=n_features)
vocab_size = len(dic.token2id) # Vocabulary size

# List of documents (each document is now a list of word indexes)
# We have removed all words not in the k most frequent words
texts_idx = [[dic.token2id[word] for word in text 
              if word in dic.token2id] for text in texts]

# Convert each index to a one hot vector
W = np.array([np.eye(vocab_size)[text] for text in texts_idx if len(text) > 0])

# Keep track of id to word mapping to visualize top words of topics
id2word = dict([[v, k] for k, v in dic.token2id.items()])

pickle.dump(W, open("W.p", "wb"))

In [114]:
n_topics = len(cats)
lda = LDA_gibbs(n_topics, id2word=id2word)
lda.fit(W)
        
visualizeTopics(lda, id2word)

0
Topic 1:
space would game team year one like time first get player think
Topic 2:
space team game would one year get time play first like new
10
Topic 1:
would game team one year get like time think hockey player space
Topic 2:
space pt 25 period 10 satellite la power first data new mission
20
Topic 1:
would game team one year get like time hockey player think season
Topic 2:
space pt 25 period satellite 10 launch nasa mission system la data
30
Topic 1:
would game team one year get like time hockey player think season
Topic 2:
space pt 25 launch period satellite 10 nasa system power mission la
40
Topic 1:
would game team one year get like time hockey player think season
Topic 2:
space launch pt 25 period satellite 10 nasa system mission power la
50
Topic 1:
would game team one year get like time hockey player think season
Topic 2:
space launch pt 25 period satellite 10 nasa system mission la power
60
Topic 1:
would team game one year get time like hockey player think season
Topic 2:
